In [1]:
import os
import pandas as pd
import subprocess
import glob
import pysam
from tqdm import tqdm
import subprocess

In [2]:
group_name = 'NP-SUB-Glut.18mo'

In [3]:
cell_type, age = group_name.split('.')
cell_type = cell_type.replace('-',' ')

In [6]:
input_dir = "/ceph/gale-1/qzeng/AmbData/CellType_Allc/"
output_dir = f"{os.getcwd()}/balanced_allc"

In [7]:
male_merged_allcs = glob.glob(f'{input_dir}/M.CellType.Age/*/*.CGN-Merge.allc.tsv.gz')
female_merged_allcs = glob.glob(f'{input_dir}/F.CellType.Age/*/*.CGN-Merge.allc.tsv.gz')

male_ct_name = sorted(list(set([path.split('/')[-2].split('.')[0] for path in male_merged_allcs])))
female_ct_name = sorted(list(set([path.split('/')[-2].split('.')[0]  for path in female_merged_allcs])))
len(male_ct_name), len(female_ct_name)

(79, 57)

In [6]:
modified_male_ct_name = [ct.replace('_',' ').replace('-',' ') for ct in male_ct_name]
modified_female_ct_name = [ct.replace('_',' ').replace('-',' ') for ct in female_ct_name]

In [7]:
male_dict = dict(zip(modified_male_ct_name, male_ct_name))
female_dict = dict(zip(modified_female_ct_name, female_ct_name))

In [8]:
male_path = f"{input_dir}/M.CellType.Age/{male_dict[cell_type]}.{age}/{male_dict[cell_type]}.{age}.CGN-Merge.allc.tsv.gz"
female_path = f"{input_dir}/F.CellType.Age/{female_dict[cell_type]}.{age}/{female_dict[cell_type]}.{age}.CGN-Merge.allc.tsv.gz"

In [9]:
male_df = pd.read_csv(male_path, sep = '\t', header = None)
female_df = pd.read_csv(female_path, sep = '\t', header = None)

In [10]:
male_df.index = male_df[0] + '-' + male_df[1].astype(str)
female_df.index = female_df[0] + '-' + female_df[1].astype(str)
#shared_cytosine_id = list(set(male_df.index) & set(female_df.index))

In [11]:
shared_cytosine_id = [index for index in male_df.index if index in female_df.index]
len(shared_cytosine_id)

19863877

In [12]:
male_df = male_df.loc[shared_cytosine_id]
female_df = female_df.loc[shared_cytosine_id]

In [13]:
male_total_cov = male_df[5].sum()
female_total_cov = female_df[5].sum()
div_by = male_total_cov/female_total_cov
div_by

2.0766901235051325

In [14]:
male_df[4] = (male_df[4] / div_by).astype('int32')
male_df[5] = (male_df[5] / div_by).astype('int32')

In [15]:
allc_df = female_df.copy()
allc_df[4] = female_df[4] + male_df[4]
allc_df[5] = female_df[5] + male_df[5]

In [16]:
save_dir = '/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/Total.Balanced.Merged.Allc'

In [17]:
allc_df.to_csv(f"{save_dir}/{group_name}.Both-Gender.CGN-Merge.allc.tsv", sep = '\t', 
               index = None, header = None)

In [21]:
subprocess.run(f"bgzip {save_dir}/{group_name}.Both-Gender.CGN-Merge.allc.tsv", 
               shell = True)

CompletedProcess(args='bgzip /home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/Total.Balanced.Merged.Allc/NP-SUB-Glut.18mo.Both-Gender.CGN-Merge.allc.tsv', returncode=0)

In [22]:
subprocess.run(f"allcools tabix-allc {save_dir}/{group_name}.Both-Gender.CGN-Merge.allc.tsv.gz", shell = True)

command	tabix-allc
allc_path	/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/Total.Balanced.Merged.Allc/NP-SUB-Glut.18mo.Both-Gender.CGN-Merge.allc.tsv.gz
reindex	False
Executing tabix-allc...
tabix-allc finished.


CompletedProcess(args='allcools tabix-allc /home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/Total.Balanced.Merged.Allc/NP-SUB-Glut.18mo.Both-Gender.CGN-Merge.allc.tsv.gz', returncode=0)